<h1> Internet Explorer 11 Functionality Job Report </h>

In [ ]:
### Set Export Folder
import pandas as pd
import numpy as np
import datetime as dt
[REDACTED]/.pyplot as plt
import seaborn as sns
import json

In [ ]:
# add current timestamp to filename for reference
current_time = (dt.datetime.utcnow().strftime('%Y_%m_%d_%H%M%S'))

# export folder will contain all csv exported DataFrames for Ticket Creation
export_folder = 'd:/exports/'

### Import Modules, Call API, Create DataFrame

In [ ]:
# import configparser for env secrets
from configparser import ConfigParser

config = ConfigParser()
config.read('d:/git/example_infrastructure_data_dev/config/env.ini')
import requests
from requests.structures import CaseInsensitiveDict

# import and assign secrets from env.ini
base_uri = config['dattormm']['base_uri']
api_key = config['dattormm']['api_key']
api_secret = config['dattormm']['api_secret']
# call token api url
token_uri = config['dattormm']['token_uri']
# construct header
headers = CaseInsensitiveDict()
headers['Content-Type'] = 'application/x-www-form-urlencoded'

# construct req body
data = CaseInsensitiveDict()
data['grant_type'] = 'password'
data['username'] = api_key
data['password'] = api_secret

# request content response
resp = requests.post(token_uri, headers=headers, data=data, auth=('public-client', 'public'))
content = resp.content.decode('utf-8')
c_dict = json.loads(content)

access_token = c_dict['access_token']
## Create Devices DataFrame
# request content response
request_url = f'{base_uri}/account/devices'

# construct header
headers = CaseInsensitiveDict()
headers['Authorization'] = f'Bearer {access_token}'
headers['Content-Type'] = 'application/json'

# construct req body
data = ''

print(f'Request URL: {request_url}')

resp = requests.get(request_url, headers=headers, data=data)
content = resp.content.decode('utf-8')
c_dict = json.loads(content)
print(c_dict['pageDetails']['nextPageUrl'])

# iterate and combine remaining pages
df_devices = pd.DataFrame(c_dict['devices'])
while c_dict['pageDetails']['nextPageUrl']:
    next_page = c_dict['pageDetails']['nextPageUrl']
    resp = requests.get(next_page, headers=headers, data=data)
    content = resp.content.decode('utf-8')
    c_dict = json.loads(content)
    print(c_dict['pageDetails']['nextPageUrl'])
    df_nextpage = pd.DataFrame(c_dict['devices'])
    df_devices = pd.concat([df_devices, df_nextpage], ignore_index=False)

## Create New Columns from Dictionary Columns

### Set Index to device UID

In [ ]:
df_devices.set_index('uid',inplace=True)

### Type | Category

In [ ]:
def device_category(device):
    try:
        return device['category']
    except:
        return None

In [ ]:
def device_type(device):
    try:
        return device['type']
    except:
        return None

In [ ]:
df_devices['category'] = df_devices['deviceType'].apply(device_category)
df_devices['type'] = df_devices['deviceType'].apply(device_type)

In [ ]:

#Rename 'type' values to split devices into (2) : 'computer' or 'server'
#df_devices['type'].replace({'Desktop':'computer','Laptop':'computer','Server':'server'},inplace=True)

In [ ]:
#df_devices.fillna(np.NAN,inplace=True)

In [ ]:
def ie11Enabled(df):
    return df['udf25']
def ie11toEdge(df):
    return df['udf26']

In [ ]:
df_devices['ie11Enabled'] = df_devices['udf'].apply(ie11Enabled)
df_devices['ie11toEdge'] = df_devices['udf'].apply(ie11toEdge)

In [ ]:
#df_devices['ie11Enabled'].replace({' ':np.NAN,None:np.NaN},inplace=True)

In [ ]:
df_devices['ie11Enabled'] = df_devices['ie11Enabled'].astype('bool')

In [ ]:
df_ie11_functionality = df_devices[['siteName','category','ie11Enabled']]


In [ ]:
df_ie11_functionality.value_counts('ie11Enabled')

In [ ]:
df_ie11_functionality

In [ ]:
df_ie11_functionality.to_csv('[REDACTED]/.csv')

In [ ]:
fig = plt.figure(figsize=(20,40))
sns.barplot(data=df_ie11_functionality_desc['count'])

In [ ]:
df_ie11_functionality.to_csv('[REDACTED]/.csv')

In [ ]:
df_ie11_functionality.info()